# Facial Recognition and Verification in a Real Time Video (Webcam)


A video by definition is collection of frames. A frame is an image. So essentially if we can recognize faces in every frame of the video, we can recognize every face in the video. 

## About the Code 

```
with open("face-labels-single.pickle", 'rb') as f:
```
First we load in the dictionary with the list of encodings and labels for verification

Next we read the frame we want to recognize into a variable and resize it to 480px(for faster processing). Then, we convert it
to an RGB format.

After that we retrieve the faces in the form of coordinates using ```face_recognition.face_locations()``` and since we no longer have only 1 image to process we cannot afford to use cnn as our model for face detection. Hence we use the relatively less accurate but still pretty good <b>hog</b>. However, if you have the resources and can run this on a GPU, go ahead and run it using  ```model="cnn"```.
The ```number_of_times_to_upsample``` parameter(optional) essentially tells the code to look for smaller faces(default is 1). Would recommend to let it be 1 as it often leads to false positives when you're trying to detect smaller faces. 


NOW, once we have all the encodings i.e all the faces detected we try and and match them with the known faces encodings

### How we do this you ask?

We compare the encoding from the image with known ones by calculating the <b>Euclidian Distance</b>

Euclidean distance is the length of a straight line between two vectors in Euclidean space. i.e. 

<img src="http://mathworld.wolfram.com/images/equations/Distance/NumberedEquation3.gif">

Once we compare them we classify them as match or not for all of the encodings through the following code

```
matches = face_recognition.compare_faces(data["encodings"],
        encoding,tolerance=0.45)
```
        
The ``` tolerance ``` acts as the threshold for comparison , if the Euclidiean distance is less than 0.45 then the encoding is classified as a match(True) else not a match(fals)

Hence matches is a grid of size (known faces * encodings found in the image). Next we parse through the grid and find out which encoding has the maximum number of matches and classify that as our found face.

Finally we take all the faces verfied and print them on the image.

#### That's it folks! We have a Real-Time Face Recognizer


## Required Libraries

In [2]:
import face_recognition 
import cv2
import pickle
import imutils

## Retrieving Trained Data via Pickle


In [5]:
# Accessing Webcam
video_capture = cv2.VideoCapture(0)
# if you wish to run on a video instead of real-time simply replace the 0 with the path of the video in ''

# Initialize some variables
names = []
process_this_frame = True

# data is a dictionary loaded with two lists
# first containing a list of all the encodings
# second containing the corresponding labels


with open("face-labels-cnn.pickle", 'rb') as f:
    data = pickle.load(f)


## Frame-wise Detection and Verification

In [6]:
while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    if ret == True:
        #convert BGR to RGB
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #resizing to 480p for a decent frame rate and fast processing
        #you can tweak it find the best fit
        
        rgb = imutils.resize(frame, width=480)
        frame = imutils.resize(frame,width=480)
        
        #face detection and encoding        
        
        boxes = face_recognition.face_locations(rgb,model="hog")
        # boxes contains the coordinates of the face(s) in the frame
        # boxes = list of (y,x+w,y+h,x) or (top,right,bottom,left)
        
        encodings = face_recognition.face_encodings(rgb, boxes)
        
        # encoding contains A list of 128-dimensional face encodings (one for each face in the image)
        # encoding is essentially a list of 128 values representing the 128 hog features of the face
        
        if  not encodings is None and boxes!=[] : # checking if a face exists in the frame
            name = "Unknown"
            for encoding,box in zip(encodings,boxes):
                # attempt to match each face in the input image to our known
                top,right,bottom,left = box
                frame = cv2.rectangle(frame, (left,top), (right, bottom), (255, 0, 0), 2) #rectange around the face
                matches = face_recognition.compare_faces(data['encodings'],encoding,tolerance=0.45)
                
                # data[encoding] is a list of encodings from the trained data 
                # that we retrived from the pickle file
                #  if two face descriptor vectors have a Euclidean
                # distance(tolerance) between them less than 0.45 then they are from the same
                # person, otherwise they are from different people.
                # By default the tolerance is set to 0.6 but I personally found
                # 0.45 to be more accurate for my dataset
                
                if True in matches : 
                    matchedIdxs = [i for (i, b) in enumerate(matches) if b]
                    counts = {}

                    # loop over the matched indexes and maintain a count for
                    # each recognized face face
                    for i in matchedIdxs:
                        name = data["names"][i]
                        counts[name] = counts.get(name, 0) + 1

                # determine the recognized face with the largest number
                # of votes (note: in the event of an unlikely tie Python
                # will select first entry in the dictionary)
                    name = max(counts, key=counts.get)
                else :
                    name = "Unknown"
                y = top - 15 if top - 15 > 15 else top + 15
                cv2.putText(frame, name, (left,y), cv2.FONT_HERSHEY_SIMPLEX, 1, 255, 2) #putting the name on the face
                names.append(name)
                #print(name)
        cv2.imshow('Video', frame)
    else: 
        break


        # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()
        
        